In [1]:
from enhancedAgent_v2 import *
from pythonTask import *

2025-06-13 04:55:28,000 - psutil_patch - INFO - psutil 已经导入，补丁不会再次导入它
2025-06-13 04:55:34,152 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [2]:
调用智能体的原则='''

### 智能体协同七诫
1. **能力透明**  
   *每个智能体必须显式声明能力边界*  
   > 教训：未验证方法存在导致流程崩溃

2. **状态显化**  
   *跨智能体数据必须通过受控总线传递*  
   > 教训：隐式变量共享引发不可测错误

3. **错误熔断**  
   *单点故障必须隔离在发生层*  
   > 教训：模型错误扩散至文件验证层

4. **验证前移**  
   *关键检查点嵌入执行流中部*  
   > 教训：后置验证增加回溯成本

5. **知识反哺**  
   *异常模式须转化为防范知识*  
   > 教训：相同错误在不同环节重复发生

6. **熵感控制**  
   *系统需持续监测并压缩认知混乱度*  
   > 教训：错误处理逻辑自身产生新熵

7. **量子信任**  
   *所有交互需携带可验证执行证明*  
   > 教训：依赖未经验证的输出导致连环失败

### 终极法则
**智能体不是工具而是生态**  
> 成功系统 = 清晰能力边界 × 严格状态管理 × 自愈机制  
> 失败系统 = 模糊责任 + 隐式依赖 + 脆弱传递  

这七条戒律构成智能体协同的免疫基因，使系统从机械执行进化为有机生命体。 
'''

In [ ]:
# 销售分析任务
# 设置代理服务器
import os
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'

# os.environ["AGENT_MAX_TOKENS"] = "1000000"
from pythonTask import *
from knowledge_agent import promgraming_knowledge

 
# llm=llm_claude_sonnet_4
llm=llm_gemini_2_5_pro_preview_06_05_google
# llm=llm_deepseek

# 实例化 MultiStepAgent_v2 时不传入 agent_specs
multi_agent = MultiStepAgent_v2(llm=llm)
multi_agent.loadKnowledge(调用智能体的原则)

# 使用 register_agent 动态注册 Agent
general_agent = Agent(llm=llm_deepseek)
general_agent.api_specification='''
general_agent,擅长执行各种任务
'''
multi_agent.register_agent(
    name="general_agent",
    instance=general_agent
)

document_agent = Agent(llm=llm_deepseek)
document_agent.loadKnowledge(promgraming_knowledge)
document_agent.loadKnowledge('如果指令要求你写文档，你应该调用gemini语言模型生成文档')
document_agent.api_specification='''
文档Agent,擅长调用gemini模型写文档
'''
multi_agent.register_agent(
    name="document_agent",
    instance=document_agent
)


# 示例主指令
# main_instruction = "请用python写一个hello world程序"
main_instruction = """

# 销售数据分析任务

sales_data.csv是销售数据文件，请使用此文件进行数据分析。

# 规则
1. 不要生成图表
2. 报告中必须包含每个地区，每个产品，每个销售人员的销售额
3. 分析报告保存到sales_analysis_report.md
4. 分析报告必须调用gemini模型生成

"""

# 执行多步骤任务
result = multi_agent.execute_multi_step(main_instruction)
print("多步骤执行结果：")
print(result)

In [ ]:
print('hello world')

In [ ]:
response=multi_agent.chat_stream('写个使用智能体的简短的经验教训总结')
for chunk in response:
    print(chunk, end='', flush=True)

In [3]:
# 示例2：翻译模式（方案2）
llm = llm_deepseek

# 创建翻译模式的多步骤智能体（方案2：支持控制流）
workflow_agent = MultiStepAgent_v2(llm=llm_gemini_2_5_pro_preview_06_05_google, use_autonomous_planning=False)

# 注册智能体
coder_agent = Agent(llm=llm)
coder_agent.api_specification = '''
通用编程智能体,擅长编写python代码
'''
workflow_agent.register_agent(
    name="coder",
    instance=coder_agent
)

test_agent = Agent(llm=llm)
test_agent.api_specification = '''
软件测试智能体,擅长运行和测试python代码
'''
workflow_agent.register_agent(
    name="tester",
    instance=test_agent
)

# 翻译模式 - 用户提供详细步骤，AI翻译为JSON计划并执行
print("=== 翻译模式示例（方案2：动态控制流）===")
result = workflow_agent.execute_multi_step("""
1. coder: 实现一个简单的计算器类，包含加减乘除功能和完整单元测试
2. coder: 把代码保存到simple_calculator.py
3. tester: 运行simple_calculator.py，执行所有测试
4. decision_maker: 分析测试结果，测试通过则完成工作流，测试失败则状态转移至步骤3重新执行测试
""")

2025-06-13 05:11:26,644 - enhancedAgent_v2 - DEBUG - 已注册 Agent: coder
2025-06-13 05:11:26,647 - enhancedAgent_v2 - DEBUG - 已注册 Agent: tester
2025-06-13 05:11:26,654 - enhancedAgent_v2 - DEBUG - 已注册 Agent: decision_maker
2025-06-13 05:11:26,657 - enhancedAgent_v2 - DEBUG - 自动注册了决策智能体


2025-06-13 05:11:26,682 - enhancedAgent_v2 - DEBUG - 从对象中解析到步骤，共 4 个步骤
2025-06-13 05:11:26,683 - enhancedAgent_v2 - DEBUG - 生成计划: [{'id': 'step1', 'name': '实现计算器类及单元测试', 'instruction': '实现一个简单的计算器类，包含加减乘除功能和完整单元测试', 'agent_name': 'coder', 'instruction_type': 'execution', 'phase': 'execution', 'expected_output': '包含计算器类和单元测试的Python代码字符串', 'prerequisites': '无', 'status': 'pending'}, {'id': 'step2', 'name': '保存代码到文件', 'instruction': '把代码保存到simple_calculator.py', 'agent_name': 'coder', 'instruction_type': 'execution', 'phase': 'execution', 'expected_output': '名为simple_calculator.py的代码文件已成功创建并包含代码', 'prerequisites': '计算器类和单元测试的代码已经实现', 'status': 'pending'}, {'id': 'step3', 'name': '运行单元测试', 'instruction': '运行simple_calculator.py，执行所有测试', 'agent_name': 'tester', 'instruction_type': 'execution', 'phase': 'verification', 'expected_output': '单元测试的执行结果，包括成功或失败的详细信息', 'prerequisites': '代码已保存到simple_calculator.py文件', 'status': 'pending'}, {'id': 'step4', 'name': '分析测试结果并决策', 'instruction': '分析测试结果

=== 翻译模式示例（方案2：动态控制流）===

当前执行计划:
[
  {
    "id": "step1",
    "name": "实现计算器类及单元测试",
    "instruction": "实现一个简单的计算器类，包含加减乘除功能和完整单元测试",
    "agent_name": "coder",
    "instruction_type": "execution",
    "phase": "execution",
    "expected_output": "包含计算器类和单元测试的Python代码字符串",
    "prerequisites": "无",
    "status": "pending"
  },
  {
    "id": "step2",
    "name": "保存代码到文件",
    "instruction": "把代码保存到simple_calculator.py",
    "agent_name": "coder",
    "instruction_type": "execution",
    "phase": "execution",
    "expected_output": "名为simple_calculator.py的代码文件已成功创建并包含代码",
    "prerequisites": "计算器类和单元测试的代码已经实现",
    "status": "pending"
  },
  {
    "id": "step3",
    "name": "运行单元测试",
    "instruction": "运行simple_calculator.py，执行所有测试",
    "agent_name": "tester",
    "instruction_type": "execution",
    "phase": "verification",
    "expected_output": "单元测试的执行结果，包括成功或失败的详细信息",
    "prerequisites": "代码已保存到simple_calculator.py文件",
    "status": "pending"
  },
  {
    "id": "step4",
    "

2025-06-13 05:11:39,246 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-13 05:11:51,262 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-13 05:12:01,132 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"
2025-06-13 05:12:11,090 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"



执行步骤 1/4: 实现计算器类及单元测试


...


决策结果: continue
原因: 步骤 'step3: 运行单元测试' 已成功执行，并且所有测试都已通过。根据工作流计划，下一步是执行 'step4: 分析测试结果并决策'，以根据测试通过的结果正式完成工作流。

执行步骤 4/4: 分析测试结果并决策


2025-06-13 05:18:11,785 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"


好的，我将调用决策智能体（decision_maker）来分析上一步的测试结果。

**分析过程如下：**

1.  **获取输入：** 决策智能体接收到上一步（step3）的测试结果。
2.  **分析结果：** 测试输出包含 `Ran 4 tests in 0.001s` 和 `OK`，这明确表示所有单元测试都已成功通过。
3.  **应用规则：** 根据指令 “分析测试结果，测试通过则完成工作流，测试失败则状态转移至步骤3重新执行测试”，当前情况满足 “测试通过” 的条件。
4.  **得出结论：** 决策智能体决定完成整个工作流。

**最终决策：**

工作流已成功达成所有目标，任务完成。

2025-06-13 05:18:22,906 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 200 OK"



决策结果: complete
原因: 所有计划的步骤都已成功执行。最后的决策步骤（step4）根据成功的单元测试结果，确认了工作流的目标已经达成。因此，整个工作流已完成。
